In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [2]:
import pandas as pd
import numpy as np
import gensim
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [3]:
df = pd.read_csv('/content/training1600000.csv',encoding = "ISO-8859-1",header=None)

In [4]:

df.columns = ["polarity",'id','date','query','user','tweet']
df

,polarity,id,date,query,user,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [5]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+',' ',text)
    text = re.sub('[^a-zA-Z]',' ',text)
    text = word_tokenize(text)
    text = [item for item in text if item not in stop_words]
    text = [lemma.lemmatize(w) for w in text]
    text = [i for i in text if len(i)>2]
    text = ' '.join(text)
    return text

In [6]:
stop_words = set(stopwords.words('english'))
lemma = WordNetLemmatizer()

In [14]:
df_new = df.sample(n=len(df))

In [15]:
df_new['polarity'].value_counts()

0    800000
4    800000
Name: polarity, dtype: int64

In [16]:
df_new['clean_tweet'] = df_new['tweet'].apply(clean_text)

In [17]:
x_train = df_new['clean_tweet']

In [18]:

tweets = df_new['clean_tweet'].tolist()
words = []
for i in range(len(tweets)):
  words.append(tweets[i].split())


In [19]:
import gensim.corpora as corpora
id2word = corpora.Dictionary(words)
texts = words
corpus = [id2word.doc2bow(text) for text in texts]

In [20]:
from pprint import pprint
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10)
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.036*"know" + 0.023*"sorry" + 0.014*"get" + 0.013*"hope" + 0.013*"hey" + '
  '0.013*"lol" + 0.012*"let" + 0.011*"help" + 0.010*"well" + 0.009*"coming"'),
 (1,
  '0.020*"happy" + 0.016*"phone" + 0.014*"hurt" + 0.012*"birthday" + '
  '0.010*"trying" + 0.009*"got" + 0.008*"cute" + 0.008*"damn" + 0.007*"head" + '
  '0.007*"face"'),
 (2,
  '0.044*"miss" + 0.024*"morning" + 0.022*"good" + 0.017*"hate" + 0.009*"eat" '
  '+ 0.009*"much" + 0.009*"missing" + 0.009*"listening" + 0.009*"food" + '
  '0.008*"thinking"'),
 (3,
  '0.030*"back" + 0.029*"going" + 0.029*"work" + 0.022*"home" + 0.021*"today" '
  '+ 0.018*"day" + 0.017*"get" + 0.017*"got" + 0.015*"tomorrow" + '
  '0.011*"getting"'),
 (4,
  '0.045*"day" + 0.030*"time" + 0.029*"night" + 0.023*"last" + 0.022*"good" + '
  '0.014*"week" + 0.013*"today" + 0.012*"one" + 0.011*"working" + '
  '0.010*"suck"'),
 (5,
  '0.076*"quot" + 0.035*"twitter" + 0.012*"tell" + 0.011*"get" + 0.011*"bored" '
  '+ 0.011*"yes" + 0.010*"please" + 0.010*"fo

In [21]:
type(doc_lda)

gensim.interfaces.TransformedCorpus

In [23]:
len(doc_lda)

1600000

In [27]:
print(lda_model.print_topics()[0])


(0, '0.036*"know" + 0.023*"sorry" + 0.014*"get" + 0.013*"hope" + 0.013*"hey" + 0.013*"lol" + 0.012*"let" + 0.011*"help" + 0.010*"well" + 0.009*"coming"')
